In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
from numpy import log as ln
from termcolor import colored
from scipy.stats.mstats import gmean 

Método que grafica un diagrama de barras

In [3]:
def bar_plot(x,name):
    plt.figure(figsize=(12,5))
    ax = plt.subplot()
    ax = sns.barplot(x=data[x].value_counts().index, y=data[x].value_counts(), data=data,edgecolor='black', color='gray')
    ax.set_xlabel(name,size=20)
    ax.set_ylabel('frecuencia ni',size=15)
    ax.set_title(f'Diagrama de barras {name}',size=15)
    plt.show()

Método que grafica una ojiva y el histograma

In [4]:
def hist_Ogive_plot(x,m,name):
    plt.figure(figsize=(25,15))
    ax = plt.subplot(231)
    sns.distplot(data[x], bins=m,color='gray', kde=False)
    ax.axvline(data[x].mean(), c='r', linestyle = '--', label = 'Promedio')
    ax.axvline(data[x].median(), c='b', linestyle = '--', label = 'Mediana')
    ax.set_title(f"Histograma de {name}",size=18)
    ax.set_ylabel('Frecuencia Absoluta ni',size=15)
    ax.set_xlabel(name,size=15)
    ax.legend()
    ax = plt.subplot(232)
    sns.kdeplot(data[x], color='gray', shade=True)
    ax.axvline(data[x].mean(), c='r', linestyle = '--', label = 'Promedio')
    ax.axvline(data[x].median(), c='b', linestyle = '--', label = 'Mediana')
    ax.set_title(f"Diagrama de densidad {name}",size=18)
    ax.set_ylabel('Frecuencia Absoluta fi',size=15)
    ax.set_xlabel(name,size=15)
    ax.legend()

    ax = plt.subplot(233)
    sns.distplot(data[x], bins=m, hist_kws=dict(cumulative=True), color='gray')
    ax.set_title(f"Diagrama de Ojiva de {name}",size=18)
    ax.set_ylabel('Freacuencia Relativa Acumulada Fi',size=15)
    ax.set_xlabel(name,size=12)

In [5]:
def box_plot_B(name,serie):
    ax = sns.boxplot(x=serie,color='red')
    ax.set_title(f'Diagrama de cajas de {name}')
    ax.set_xlabel(name)

Grafica un digrama de caja

In [6]:
def box_plot(x,name):
    plt.figure(figsize=(10,5))
    ax = plt.subplot()
    ax = sns.boxplot(x=data[x], color='gray')
    ax.set_title(f'Diagrama de cajas de {name}',size=20)
    ax.set_xlabel(name,size=15)

Determina el número de intervalos que debe tener la tabla de frecuencias

In [7]:
def intervals_class(N):
    m1 = np.sqrt(N)
    m2 = 1+3.3*np.log10(N)
    m3 = ln(N)/ln(2)
    return [m1,m2,m3] 

Método que crea los intervalos que debe tener la tabla de frecuencias

In [8]:
def made_intervals_class(x, m,date,t):
    range_i =  date[x].max() - date[x].min()
    amplitude = range_i/m
    tupla = (range_i,np.rint(amplitude)) 
    if t == 1:
        tupla = (range_i,np.round(amplitude,5))
    return tupla

Método que crea la marca de clase

In [9]:
def class_mark(intervale,c):
    f_value = np.round((intervale[0][1]+intervale[0][0])/2,5)
    m_values = []
    m_values.append(f_value)
    for i in range(len(intervale)-1):
        f_value = np.round(f_value + c,5)
        m_values.append(f_value)
    return m_values

Crea tabla de frecuencia para datos cuantitativos

In [10]:
def made_frequency_table_A(x, date,t,name):
    
    m = int(np.rint(np.array(intervals_class(date[x].shape[0])).min()))
    temp = made_intervals_class(x, m,date,t)
    r = temp[0]
    c = temp[1]
    intervale = []
    n = data[x].min()
    for i in range(m):
        if i < m-1:
            intervale.append((n,np.around(n+c,5)))
            n = np.around(n+c,5)
        else:
            intervale.append((n,np.around(n+c,5)+(data[x].max()-(np.around(n+c,5)))))
    
    alfa = date[x]
    alfa = alfa.sort_values()
    
    ni = []
    for i in range(m):
        if i==0:
            a = alfa[alfa <= intervale[i][1]]
            ni.append(len(a))
        else:
            a = alfa[(alfa > intervale[i][0]) & (alfa<=intervale[i][1])]
            ni.append(len(a))
    dic = {name:intervale,'ni':ni}
    dic = pd.DataFrame(dic)
    total = dic['ni'].sum()
    dic['fi'] = dic['ni']/total
    dic['Ni'] = np.cumsum(dic['ni'])
    dic['Fi'] = np.cumsum(dic['fi'])
    dic['Xí'] = class_mark(intervale,c)
    dic.loc[str(m)] = ['Total',total,' ',' ',' ',' ']
    return dic    

Crea tabla de frecuencia para datos categoricos

In [11]:
def made_frequency_table_B(x,date,m):
    
    intervale = []
    ni = []
    intervale = list(date[x].unique())
    ni = list(date[x].value_counts(dropna = False).values)
    dic = pd.DataFrame()
    total = np.sum(ni)
    dic[m] = intervale
    dic['ni'] = ni
    dic['fi'] = np.round(dic['ni']/total,5)
    dic['Ni'] = np.cumsum(dic['ni'])
    dic['Fi'] = np.cumsum(dic['fi'])
    dic.loc[len(ni)+1] = ['Total',total,' ',' ',' ']
    return dic

Calcula el coeficiente de asimetria y de curtosis de la distribución

In [12]:
def is_asimetric_curtosis(columna):
    asimetria = columna.skew()
    curtosis = columna.kurt()
    message = ''
    message_2 = ''
    if asimetria == 0:
        message = 'Asimetrica'
    elif asimetria > 0:
        message = 'Asimetrica con cola hacia la derecha'
    else:
        message = 'Asimetrica con cola hacia la izquierda'
    
    if curtosis == 3:
        message_2 = 'Mesocúrtica'
    elif asimetria > 3:
        message_2 = 'Leptocúrtica'
    else:
        message_2 = 'Platicúrtica'
    return asimetria, curtosis, message, message_2

In [13]:
print(colored(f'Intaciamos data data frame de los indicadores para datos', attrs=['bold']))
columns = ['X','Me','Mg','Mo','Fr','R','Min','Max','§2','S','CV','Q1','Q2','Q3','Ls','Li','RIC','Outliers','per_out','A','CK']
indices = ['type','pr','flg', 'cryptocurrency','td','sp', 'dp', 'ipkt', 'ibyt']
features_No_Agrupadas = pd.DataFrame(0,columns = columns, index=indices)
features_Agrupadas =  pd.DataFrame(0,columns = columns, index=indices)

Intaciamos data data frame de los indicadores para datos


In [14]:
def summary_descriptivo(ind,col,val):
    features_No_Agrupadas.loc[ind,col] = val

In [15]:
def summary_descriptivo_Agrupy(ind,col,val):
    features_Agrupadas.loc[ind,col] = val

# Preparación de los datos
#### Descripción de las caracteristicas ('features')
- ts:     Hora de inicio del flujo
- te:     Hora de finalización del flujo
- td:     Duración del flujo
- sa:     Dirección IP origen
- da:     Dirección IP de destino
- sp:     Puerto de origen
- dp:     Puerto de destino
- pr:     Protocolo
- flg:    Banderas
- fwd:    Estado de reenvío
- stos:   Tipo de servicio de origen
- ipkt:   Paquetes de entrada
- ibyt:   Bytes de entrada
- opkt:   Paquetes de salida
- obyt:   Bytes de salida
- in:     Interfaz de entrada
- out:    Interfaz de salida
- sas:    Número de sistema autónomo BGP de origen
- das:    Número de sistema autónomo BGP de destino
- smk:    Máscara de red de origen
- dmk:    Máscara de red de destino
- dtos:   Tipo de servicio de destino
- dir:    Dirección del flujo
- nh:     Enrutador de siguiente salto
- nhb:    Enrutador de siguiente salto BGP
- svln:   VLAN de origen
- dvln:   VLAN de destino
- ismc:   Dirección MAC de la fuente de entrada   
- odmc:   Dirección MAC de destino de salida
- idmc:   Introduzca la dirección MAC de destino
- osmc:   Dirección MAC de la fuente de salida
- mpls1:  Etiqueta MPLS 1
- mpls2:  Etiqueta MPLS 2
- mpls3:  Etiqueta MPLS 3
- mpls4:  Etiqueta MPLS 4
- mpls5:  Etiqueta MPLS 5
- mpls6:  Etiqueta MPLS 6
- mpls7:  Etiqueta MPLS 7
- mpls8:  Etiqueta MPLS 8
- mpls9:  Etiqueta MPLS 9
- mpls10: Etiqueta MPLS 10
- cl:     Latencia del cliente
- sl:     Latencia del servidor
- al:     Latencia de la aplicación
- ra:     Exportando la dirección IP del sistema (enrutador)
- eng:    Tipo / ID de motor
- exid:   Exportando el ID del sistema
- tr:     Marca de tiempo de flujo recibido
- type:   Tipo de registro Mina/ no Mina 
- cryptocurrency: : Tipo criptomoneda
- id_Cryptocurrency:: Id Criptomoneda

In [16]:
print(colored(f'Cargamos los datos', attrs=['bold']))
data = pd.read_csv('./dataA.csv')
data.head()

Cargamos los datos


FileNotFoundError: [Errno 2] File b'./dataA.csv' does not exist: b'./dataA.csv'

# Análisis inicial

In [ ]:
print(colored(f'Observamos el número de registros en total, cuantos valores hay por columnas, así como sus valores nulos y el tipo de dato de cada columna', attrs=['bold']))
data.info()

In [ ]:
print(colored(f'El conjunto de datos consta de {len(data.index)} registros y {len(data.columns)} columnas', attrs=['bold']))

### Inspección datos faltantes

In [ ]:
print(colored(f'Valores faltantes por columnas, (False: Valores no nulos), (True: Valores nulos)', attrs=['bold']))
avaliable = data.isnull()
avaliable.apply(pd.Series.value_counts).T

In [ ]:
d = ['te', 'td', 'sa', 'da', 'sp', 'dp', 'pr','flg', 'fwd', 'stos', 'ipkt', 'ibyt', 'opkt', 'obyt', 'in', 'out','sas', 'das', 'smk', 'dmk', 'dtos', 'dir', 'nh', 'nhb', 'svln', 'dvln','ismc', 'odmc', 'idmc', 'osmc', 'mpls1', 'mpls2', 'mpls3', 'mpls4',
'mpls5', 'mpls6', 'mpls7', 'mpls8', 'mpls9', 'mpls10', 'cl', 'sl', 'al','ra', 'eng', 'exid', 'tr']

In [ ]:
data_concidencias = []
for i in range(len(d)):
    if i < len(d)-1:
        v = (data[data[d[i]].isnull()].index == data[data[d[i+1]].isnull()].index).all()
        t = (d[i],d[i+1], v)
        data_concidencias.append(t)

print(colored(f'Los valores nulos se distribullen en los mismos registros', attrs=['bold']))
data_concidencias

In [ ]:
avaliable =  data.notnull()
avaliable = avaliable.all(axis=1).value_counts()

In [ ]:
print(colored(f'Eliminamos los datos faltantes, ya que se concentran en los mismos registros y tan solo representan un {(avaliable[0]/avaliable[1])*100}% de los datos. Quedaron {data.shape[0] - avaliable[0]} registros', attrs=['bold']))

In [ ]:
data = data.dropna()

### Inspección de tipo de dato incorrecto
Analizo que las columnas tuvieran un tipo de dato acorde a su naturaleza. Me concentrare en datos que tengan por lo menos dos valores en los registros, puesto que hay varias categorías con un solo valor.

<font color="blue"><b><h2> Análisis</h2> </font>
Las variables: puerto de origen (sp), puerto de destino (dp), paquetes de entrada(ipkt) y número de bytes de entrada (ibyte), son de naturaleza discreta y no continuas. Por el motivo anterior le cambio el tipo de dato a un discreto.

In [ ]:
data = data.astype({'sp':'int64'})
data = data.astype({'dp':'int64'})
data = data.astype({'ipkt':'int64'})
data = data.astype({'ibyt':'int64'})

### Estructuración del conjunto de datos
Agrupo las columnas de tipo discretas(int64), continuas(float64) y categóricas(object) por separado, para hacer una análisis más organizado

In [ ]:
data_numeric_int = (data.dtypes == "int64")
data_numeric_float = (data.dtypes == "float64")
data_object  = (data.dtypes == "object")
data_numeric_int = [c for c in data_numeric_int.index if data_numeric_int[c]]
data_numeric_float = [c for c in data_numeric_float.index if data_numeric_float[c]]
data_object = [c for c in data_object.index if data_object[c]]

Variables cuantitativas discretas

In [ ]:
data_numeric_int

Variables cuantitativas continuas

In [ ]:
print(data_numeric_float)

Variables categoricas

In [ ]:
print(data_object)

### Balance del conjunto de datos
Analizare si la data esta balanceada

In [ ]:
ax = sns.countplot(x='type', data=data, color='gray')
ax.set_title("Diagrama de barras del Tipo de trafico")
ax.set_xlabel('Tipo de trafico')
ax.set_ylabel('Cantidad de observaciones')

In [ ]:
print(colored(f'Hacemos una copia del conjunto de datos original, para conservar la integridad de los datos originales', attrs=['bold']))
dataOr = data

<font color="Red"><b><h2> Análisis</h2> </font>
El conjunto de datos A como se aprecia no esta balanceada, esto es debido a que en un principio se trato de balancear el conjunto de datos B que tiene trafico minero no identificado, por tal razón el conjunto benigno se le añadió más registros. **Para remediar lo anterior, agrupare de manera pseudoaletoria registros no mineros con la finalidad de balancear el conjunto de datos A**.

In [ ]:
data_M = data[data['type'] == 'mine']
data_NM = data[data['type'] == 'not_mine']
dataNM_i = list(data_NM.index)
random.shuffle(dataNM_i)
ind_G = dataNM_i[0:data_M.shape[0]]

In [ ]:
print(colored(f'Concatenamos', attrs=['bold']))
data_agrupada = data_NM[data_NM.index.isin(ind_G)]
data = pd.concat([data_agrupada,data_M], axis=0)
print(colored(f'Genero un rango de valores', attrs=['bold']))
indices = list(data.index)
print(colored(f'El rango de valores se le aplico aleatoriedad de manera pseudoaleatoria', attrs=['bold']))
random.shuffle(indices)
print(colored(f'Reacomodo los registros del conjunto con los indices aleatorios', attrs=['bold']))
data = data.reindex(indices)
data.head()

In [ ]:
ax = sns.countplot(x='type', data=data, color='gray')
ax.set_title("Diagrama de barras del Tipo de trafico")
ax.set_xlabel('Tipo de trafico')
ax.set_ylabel('Cantidad de observaciones')

In [ ]:
print(colored(f'El conjunto de datos agrupados quedo con {data.shape[0]} registros, con respecto al original perdimos un {(data.shape[0]/dataOr.shape[0])*100}% con respecto al conjunto de datos original. ', attrs=['bold']))

# Inspección de datos no agrupados

### Datos categoricos 

No tengo en cuenta a 'references', ya que solo hace referencia a los nombres de los flujos. La moda(top) indica que hay elementos con la misma referencia, dado que concatenamos varios registros de diferentes conjuntos de datos, pero no significa que los registros sean repetidos.

In [ ]:
print(colored(f'Variables categoricas', attrs=['bold']))
data[data_object].describe().T

In [ ]:
print(colored(f'Agregue el indicador de Moda de mis variables a mi tabla de resumen', attrs=['bold']))

summary_descriptivo('type','Mo',data['type'].mode()[0])
summary_descriptivo('pr','Mo',data['pr'].mode()[0])
summary_descriptivo('flg','Mo',data['flg'].mode()[0])
summary_descriptivo('cryptocurrency','Mo',data['cryptocurrency'].mode()[0])

<font color="Red"><b><h2> Análisis</h2> </font>
- Las Hora de inicio (ts) y la hora final(te) tienen la misma moda, con la misma frecuencia.
- La dirección ip de origen (sa) y la dirección ip de origen (da) tienen la misma moda. Con frecuencias distintas
- La moda en el protocolo es TCP.
- La flag con mas iteraciones es __A__.
- Tanto el enrutador de salto y de salto BGP tienen la misma intrfaz para todos sus registros.
- La dirección Mac (idmc) de entrada y la dirección mac(odmc) no fueron registradas por la herramienta

### Datos continuos

In [ ]:
print(colored(f'Variables continuas', attrs=['bold']))
data[data_numeric_float].describe().T

<font color="Red"><b><h2> Análisis</h2> </font>

- La variable de tiempo es la unica variable, cuyo valor es distinto a 0
- La mayoria de los datos se encuentran entre el cuartil 2 y el cuartil 3

### Datos discretos

In [ ]:
print(colored(f'Variables continuas', attrs=['bold']))
data[data_numeric_int].describe().T

<font color="Red"><b><h2> Análisis</h2> </font>

- No tomo encuenta la columna index, puesto que solo hace referencia a los indices
- Puerto de origen (sp) tiene la mayoria de datos distribuidos en el segundo y tercer cuartil
- Puerto de destino (dp) tiene la mayoria de los datos en el primer y tercer cuartil, mayoritariamente en el tercero
- El flujo de paquetes parece tener la mayoria de datos por encima del tercer cuartil
- Flujo de bytes tambien parece tener la mayoria de datos por encima del tercer cuartil

# Datos atipicos

In [ ]:
data_numeric_int_list = ['sp', 'dp', 'ipkt', 'ibyt']
name = ['Puerto origen', 'Puerto destino','Paquetes de entrada','Bytes de entrada']

In [ ]:
def box_plot(x,name):
    plt.figure(figsize=(12,5))
    ax = plt.subplot()
    ax = sns.boxplot(x=data[x], color='gray')
    ax.set_title(f'Diagrama de cajas de {name}',size=20)
    ax.set_xlabel(name,size=15)

### Variables discretas

In [ ]:
m = data_numeric_int_list[0]
box_plot(m,name[0])
q1 = data[m].quantile(0.25)
q3 = data[m].quantile(0.75)
q2 = data[m].quantile(0.5)
ric = q3-q1
borde_s = q3 + 1.5*ric
borde_i = q1-1.5*ric
atip_s = data[data[m]>borde_s].shape[0]
atip_i = data[data[m]<borde_i].shape[0]
porc = ((atip_s+atip_i)/data[m].shape[0]) * 100


summary_descriptivo(m,'Q1',q1)
summary_descriptivo(m,'Q2',q2)
summary_descriptivo(m,'Q3',q3)
summary_descriptivo(m,'Ls',borde_s)
summary_descriptivo(m,'Li',borde_i)
summary_descriptivo(m,'RIC',ric)
summary_descriptivo(m,'Outliers', (atip_i+atip_s))
summary_descriptivo(m,'per_out',porc)


print(colored(f'Hay {atip_s} valores atipicos en el limite superior', attrs=['bold']))
print(colored(f'Hay {atip_i} valores atipicos en el limite inferior', attrs=['bold']))
print(colored(f'Hay {porc}% datos atipicos', attrs=['bold']))

In [ ]:
m = data_numeric_int_list[1]
box_plot(m,name[1])
q1 = data[m].quantile(0.25)
q3 = data[m].quantile(0.75)
ric = q3-q1
borde_s = q3 + 1.5*ric
borde_i = q1-1.5*ric
atip_s = data[data[m]>borde_s].shape[0]
atip_i = data[data[m]<borde_i].shape[0]
porc = ((atip_s+atip_i)/data[m].shape[0]) * 100 

summary_descriptivo(m,'Q1',q1)
summary_descriptivo(m,'Q2',q2)
summary_descriptivo(m,'Q3',q3)
summary_descriptivo(m,'Ls',borde_s)
summary_descriptivo(m,'Li',borde_i)
summary_descriptivo(m,'RIC',ric)
summary_descriptivo(m,'Outliers', (atip_i+atip_s))
summary_descriptivo(m,'per_out',porc)

print(colored(f'Hay {atip_s} valores atipicos en el limite superior', attrs=['bold']))
print(colored(f'Hay {atip_i} valores atipicos en el limite inferior', attrs=['bold']))
print(colored(f'Hay {porc}% datos atipicos', attrs=['bold']))

In [ ]:
m = data_numeric_int_list[2]
box_plot(m,name[2])
q1 = data[m].quantile(0.25)
q3 = data[m].quantile(0.75)
ric = q3-q1
borde_s = q3 + 1.5*ric
borde_i = q1-1.5*ric
atip_s = data[data[m]>borde_s].shape[0]
atip_i = data[data[m]<borde_i].shape[0]
porc = ((atip_s+atip_i)/data[m].shape[0]) * 100 

summary_descriptivo(m,'Q1',q1)
summary_descriptivo(m,'Q2',q2)
summary_descriptivo(m,'Q3',q3)
summary_descriptivo(m,'Ls',borde_s)
summary_descriptivo(m,'Li',borde_i)
summary_descriptivo(m,'RIC',ric)
summary_descriptivo(m,'Outliers', (atip_i+atip_s))
summary_descriptivo(m,'per_out',porc)

print(colored(f'Hay {atip_s} valores atipicos en el limite superior', attrs=['bold']))
print(colored(f'Hay {atip_i} valores atipicos en el limite inferior', attrs=['bold']))
print(colored(f'Hay {porc}% datos atipicos', attrs=['bold']))

In [ ]:
m = data_numeric_int_list[3]
box_plot(m,name[3])
q1 = data[m].quantile(0.25)
q3 = data[m].quantile(0.75)
ric = q3-q1
borde_s = q3 + 1.5*ric
borde_i = q1-1.5*ric
atip_s = data[data[m]>borde_s].shape[0]
atip_i = data[data[m]<borde_i].shape[0]
porc = ((atip_s+atip_i)/data[m].shape[0]) * 100 

summary_descriptivo(m,'Q1',q1)
summary_descriptivo(m,'Q2',q2)
summary_descriptivo(m,'Q3',q3)
summary_descriptivo(m,'Ls',borde_s)
summary_descriptivo(m,'Li',borde_i)
summary_descriptivo(m,'RIC',ric)
summary_descriptivo(m,'Outliers', (atip_i+atip_s))
summary_descriptivo(m,'per_out',porc)

print(colored(f'Hay {atip_s} valores atipicos en el limite superior', attrs=['bold']))
print(colored(f'Hay {atip_i} valores atipicos en el limite inferior', attrs=['bold']))
print(colored(f'Hay {porc}% datos atipicos', attrs=['bold']))

In [ ]:
data_numeric_float_list = ['td']
name = ['Duración del flujo']

### Variable continua

In [ ]:
m = data_numeric_float_list[0]
box_plot(m,name[0])
q1 = data[m].quantile(0.25)
q3 = data[m].quantile(0.75)
ric = q3-q1
borde_s = q3 + 1.5*ric
borde_i = q1-1.5*ric
atip_s = data[data[m]>borde_s].shape[0]
atip_i = data[data[m]<borde_i].shape[0]
porc = ((atip_s+atip_i)/data[m].shape[0]) * 100 

summary_descriptivo(m,'Q1',q1)
summary_descriptivo(m,'Q2',q2)
summary_descriptivo(m,'Q3',q3)
summary_descriptivo(m,'Ls',borde_s)
summary_descriptivo(m,'Li',borde_i)
summary_descriptivo(m,'RIC',ric)
summary_descriptivo(m,'Outliers', (atip_i+atip_s))
summary_descriptivo(m,'per_out',porc)

print(colored(f'Hay {atip_s} valores atipicos en el limite superior', attrs=['bold']))
print(colored(f'Hay {atip_i} valores atipicos en el limite inferior', attrs=['bold']))
print(colored(f'Hay {porc}% datos atipicos', attrs=['bold']))

# Análisis datos Agrupados

### Datos categoricos

- No tengo en cuenta a las variables 'nh', 'nhb', 'ismc', 'odmc', 'idmc', 'osmc', 'mpls1', 'mpls2', 'mpls3', 'mpls4', 'mpls5', 'mpls6', 'mpls7', 'mpls8', 'mpls9', 'mpls10', 'ra', 'eng' y 'tr' en esta etapa del análisis, puesto que solamente hay un valor y es el mismo para todos los registros.

In [ ]:
data_object_list = ['pr','flg', 'cryptocurrency', 'type']
tables_frequency_cualitative = []
namess = ['Protocolo','Banderas','Criptomonedas','Tipo']
for j, i in enumerate(data_object_list):
    tables_frequency_cualitative.append(made_frequency_table_B(i, data,namess[j]))

In [ ]:
print(colored(f'Tabla de frecuencia {namess[0]}',attrs=['bold']))
table_f = pd.DataFrame(tables_frequency_cualitative[0])
display(table_f)
bar_plot(data_object_list[0],namess[0])
v = data[data_object_list[0]].value_counts()

print(v)
print(colored(f'El protocolo con mas observaciones (Moda) en el conjunto de datos es el {v.index[0]} con {v[0]} registros.',attrs=['bold']))

In [ ]:
print(colored(f'Tabla de frecuencia {namess[1]}',attrs=['bold']))
table_f = pd.DataFrame(tables_frequency_cualitative[1])
display(table_f)
bar_plot(data_object_list[1],namess[1])
v = data[data_object_list[1]].value_counts()
print(v)
print(colored(f'El protocolo con mas observaciones (Moda) en el conjunto de datos es el {v.index[0]} con {v[0]} registros.',attrs=['bold']))

In [ ]:
print(colored(f'Tabla de frecuencia {namess[2]}',attrs=['bold']))
table_f = pd.DataFrame(tables_frequency_cualitative[2])
display(table_f)
bar_plot(data_object_list[2],namess[2])
v = data[data_object_list[2]].value_counts()
print(v)
print(colored(f'El protocolo con mas observaciones (Moda) en el conjunto de datos es el {v.index[0]} con {v[0]} registros.',attrs=['bold']))

In [ ]:
print(colored(f'Tabla de frecuencia {namess[3]}',attrs=['bold']))
table_f = pd.DataFrame(tables_frequency_cualitative[3])
display(table_f)
bar_plot(data_object_list[3],namess[3])
v = data[data_object_list[3]].value_counts()
print(v)
print(colored(f'El protocolo con mas observaciones (Moda) en el conjunto de datos es el {v.index[0]} con {v[0]} registros.',attrs=['bold']))

### Datos continuos

Las variables 'fwd', 'stos', 'opkt', 'obyt', 'in', 'out', 'sas', 'das', 'smk', 'dmk', 'dtos', 'dir', 'svln', 'dvln', 'cl', 'sl','al' y 'exid' son descartadas para esta etapa del analisis exploratorio, puesto que su valor es 0 en sus registros.

In [ ]:
data_numeric_float_list = ['td']
name_f = ['Duración del flujo']
tables_frequency_fl = []
for j, i in enumerate(data_numeric_float_list):
    tables_frequency_fl.append(made_frequency_table_A(i, data,1, name_f[j]))

In [ ]:
m = intervals_class(data.shape[0])
m = int(np.rint(np.min(m)))
m

In [ ]:
print(colored(f'Tabla de frecuencia de {name_f[0]}',attrs=['bold']))
table_f = pd.DataFrame(tables_frequency_fl[0])
display(table_f)
hist_Ogive_plot(data_numeric_float_list[0],m,name_f[0])
fila = data[data_numeric_float_list[0]]
asimetria, curtosis, message, message_2 = is_asimetric_curtosis(fila)
print(colored(f'El coeficiente de asimetria es de {asimetria}, la distribución es {message}.',attrs=['bold']))
print(colored(f'El coeficiente de curtosis es de {curtosis}, la distribución es {message_2}.',attrs=['bold']))

### Datos discretos

In [ ]:
data_numeric_int_list = ['sp', 'dp', 'ipkt', 'ibyt']        
tables_frequency_int = []
name = ['Puerto origen', 'Puerto destino','Paquetes de entrada','Bytes de entrada']
for j,i in enumerate(data_numeric_int_list):
    tables_frequency_int.append(made_frequency_table_A(i, data,1,name[j]))

In [ ]:
print(colored(f'Tabla de frecuencia de {name[0]}',attrs=['bold']))
table_f = pd.DataFrame(tables_frequency_fl[0])
display(table_f)
hist_Ogive_plot(data_numeric_int_list[0],m,name[0])
fila = data[data_numeric_int_list[0]]
asimetria, curtosis, message, message_2 = is_asimetric_curtosis(fila)
print(colored(f'El coeficiente de asimetria es de {asimetria}, la distribución es {message}.',attrs=['bold']))
print(colored(f'El coeficiente de curtosis es de {curtosis}, la distribución es {message_2}.',attrs=['bold']))

In [ ]:
print(colored(f'Tabla de frecuencia de {name[1]}',attrs=['bold']))
table_f = pd.DataFrame(tables_frequency_int[1])
display(table_f)
hist_Ogive_plot(data_numeric_int_list[1],m,name[1])
fila = data[data_numeric_int_list[1]]
asimetria, curtosis, message, message_2 = is_asimetric_curtosis(fila)
print(colored(f'El coeficiente de asimetria es de {asimetria}, la distribución es {message}.',attrs=['bold']))
print(colored(f'El coeficiente de curtosis es de {curtosis}, la distribución es {message_2}.',attrs=['bold']))

In [ ]:
print(colored(f'Tabla de frecuencia de {name[2]}',attrs=['bold']))
table_f = pd.DataFrame(tables_frequency_int[2])
display(table_f)
hist_Ogive_plot(data_numeric_int_list[2],m,name[2])
fila = data[data_numeric_int_list[2]]
asimetria, curtosis, message, message_2 = is_asimetric_curtosis(fila)
print(colored(f'El coeficiente de asimetria es de {asimetria}, la distribución es {message}.',attrs=['bold']))
print(colored(f'El coeficiente de curtosis es de {curtosis}, la distribución es {message_2}.',attrs=['bold']))

In [ ]:
print(colored(f'Tabla de frecuencia de {name[3]}',attrs=['bold']))
table_f = pd.DataFrame(tables_frequency_int[3])
display(table_f)
hist_Ogive_plot(data_numeric_int_list[3],m,name[3])
fila = data[data_numeric_int_list[3]]
asimetria, curtosis, message, message_2 = is_asimetric_curtosis(fila)
print(colored(f'El coeficiente de asimetria es de {asimetria}, la distribución es {message}.',attrs=['bold']))
print(colored(f'El coeficiente de curtosis es de {curtosis}, la distribución es {message_2}.',attrs=['bold']))

<font color='red'><h1>Resumen</h1></font>

### Diccionario de datos
- $\overline{X}$ = Media Aritmetica
- Me: Mediana
- Mg: Media Geometrica
- Mo: Moda
- Fr: Frecuencia de la moda
- R: Rango
- Min: Minimo
- Max: Maximo
- $\S^{2}$: Varianza
- S: Desviación
- CV: Coeficiente de variación
- Q1: Cuartil 1
- Q2: Cuartil 2
- Q3: Cuartil 3
- Ls: Limite Superior
- Li: Limite Inferior
- RIC: Rango intercuartilico
- Outliers: Datos atipicos
- A: Coeficiente de simetria
- CK: Coeficiente de curtosis

In [ ]:
features_No_Agrupadas.T

In [ ]:
data.to_csv('DataA_Tratada.csv',index=False)